# R-Multiples and Risk/Reward Analysis with Python

## Dev env configuration

In [1]:
# install the required python packages
# !pip install numpy pandas

## Imports

In [2]:
# import the necessary packages
from typing import Optional
from typing import Union
from typing import List
from dataclasses import dataclass
import pandas as pd

## Implementing a simple risk/reward calculator

In [3]:
@dataclass(frozen=True)
class TradeRiskParams:
    # define the data schema for risk/reward parameters for a given trade
    amount_to_risk: float
    percent_risk: float
    shares_to_buy: float
    total_investment: float

In [4]:
@dataclass(frozen=True)
class RLevelOutput:
    # define the data schema for R-level outputs
    r_level: int
    potential_pl: float
    price: float

In [5]:
class RiskRewardCalculator:

    # define the list of default R-values to compute
    DEFAULT_R_VALUES = [1, 2, 3, 4, 5]

    def __init__(
            self,
            total_account_value: float,
            buy_point: float,
            stop_loss: float,
            risk_rate: float = 0.01
    ) -> None:
        # store the total account value, buy point, stop loss, and risk rate
        self.total_account_value = total_account_value
        self.buy_point = buy_point
        self.stop_loss = stop_loss
        self.risk_rate = risk_rate

        # calculate the amount to risk on each trade
        self.amount_to_risk = self.total_account_value * self.risk_rate

        # determine the price difference between the buy point and stop loss,
        # then derive the number of shares to buy based on the amount to risk
        self.price_delta = self.buy_point - self.stop_loss
        self.shares_to_buy = self.amount_to_risk / self.price_delta

        # calculate the total amount of money to be invested based on the
        # number of shares to buy and the buy point
        self.total_investment = self.shares_to_buy * self.buy_point

        # derive the overall risk percentage of the account
        amount_ratio = self.amount_to_risk / self.total_account_value
        self.risk_pct_of_account = amount_ratio * 100

    def get_risk_parameters(self) -> TradeRiskParams:
        # construct and return the calculated risk parameters
        return TradeRiskParams(
            amount_to_risk=self.amount_to_risk,
            percent_risk=self.risk_pct_of_account,
            shares_to_buy=self.shares_to_buy,
            total_investment=self.total_investment
        )

    def r_levels(
            self,
            r_levels: Optional[List[Union[int, float]]] = None
    ) -> List[RLevelOutput]:
        # check to see if risk/reward levels were not provided
        if not isinstance(r_levels, list):
            # use the default risk/reward levels
            r_levels = self.DEFAULT_R_VALUES

        # initialize the list of R-level outputs
        results = []

        # loop over the levels
        for r in r_levels:
            # calculate (1) the potential profit/loss, and (2) the price at
            # the current level
            potential_pl = r * self.amount_to_risk
            price = self.buy_point + (r * self.price_delta)

            # update the results list
            results.append(RLevelOutput(
                r_level=r,
                potential_pl=potential_pl,
                price=price
            ))

        # return the list of r-level results
        return results

    def r_level_at_price(self, price: float) -> float:
        # compute and return the r-level for the input price
        return (price - self.buy_point) / self.price_delta

## Using our risk/reward calculator in real-world trading situations

In [6]:
# initialize the risk/reward calculator for a potential trade
calc = RiskRewardCalculator(
    total_account_value=25_000,
    buy_point=100.61,
    stop_loss=95.0
)

In [7]:
# grab the risk parameters
risk_params = calc.get_risk_parameters()

# construct a dataframe from the parameters and display it
df = pd.DataFrame({
    "Amount To Risk": [risk_params.amount_to_risk],
    "Risk % of Account": [risk_params.percent_risk],
    "Shares to Buy": [risk_params.shares_to_buy],
    "Total Investment": [risk_params.total_investment],
}, index=["Trade"]).T
df

Trade
Amount To Risk      250.000000
Risk % of Account     1.000000
Shares to Buy        44.563280
Total Investment   4483.511586

In [8]:
# calculate the risk/reward levels
r_levels = calc.r_levels()

# construct a dataframe from the R-levels and display it
df = pd.DataFrame({
    "Potential P&L": [r.potential_pl for r in r_levels],
    "Price at R": [r.price for r in r_levels],
}, index=[r.r_level for r in r_levels])
df

Potential P&L  Price at R
1          250.0      106.22
2          500.0      111.83
3          750.0      117.44
4         1000.0      123.05
5         1250.0      128.66

In [9]:
# given an input price, determine what the R-level would be
calc.r_level_at_price(130.0)

5.238859180035651